In [1]:
import pandas as pd
import json
import requests
import bs4
import csv

from geopy.geocoders import Nominatim
import os.path
from os import path

# initialize Nominatim API  
geolocator = Nominatim(user_agent="geoapiExercises")




In [2]:
""" Analyse de la page HTML atlashymenoptera.net pour récupérer les liens de toutes les sources de données pour les différentes espèces """

def get_all_data_sources():
    # html page bombus dataset
    page_url = 'http://www.atlashymenoptera.net/page.aspx?id=169'
    page_text = requests.get(page_url).text
    soup = bs4.BeautifulSoup(page_text, 'html.parser')
    
    data_sources = []

    # Analyse des éléments de DOM ayant la classe mt-3
    for row in soup.find_all(class_='mt-3'):
        # Le nom de l'espèce est contenu dans l'élément de classe nomtaxon
        specie_name = row.find(class_='nomtaxon')
        if specie_name is None: continue
        # Le lien vers la source de données est contenu dans un sous-élements <a> du sous-éléement de classe col-lg-2
        link = row.find(class_='col-lg-2').find('a', text='Download dataset')
        if link is None: continue
        # Enlever les cactères blancs sur le nom de l'espèce
        specie_name = specie_name.text.strip()
        # Récupération des liens vers les images : ils sont contenus dans des sous-éléments de classe mb-3
        next_row = row.findNext(class_='mb-3').find_all('img')
        imgs = [nr.get('src') for nr in next_row]
        # Lien de sources rattachée à une espèce (identifiant espèce + nom de l'espèce + lien images + liens CSV )
        data_sources.append({
            'link': link.get('href'),
            'imgs': imgs,
            'name': specie_name,
            'id': len(data_sources) + 1
        })

    return data_sources

In [3]:
"""  # Affichage de la répartition par année (pour debug) """
def displayContentByYear(cleaned_dataset):
    testByYear = {}
    print(testByYear.keys())
    for row in cleaned_dataset:
        year1 = row[0] + "#" + row[3]
        year1 = row[0]
        #print("year1 : ", year1)
        if(year1 in testByYear.keys()):
            testByYear[year1]+= 1
        else:
            testByYear[year1] = 1
        #print(row)
    testByYear2 = sorted(testByYear.items(), key=lambda kv: -1*kv[1])
    print(testByYear2)


def fetch_all_dataset(data_sources):
    # données de l'ensemble des espèces
    cleaned_dataset = []
    # Parccourrir les liens que l'on a sur les différentes espèces
    for source in data_sources:
        if(1*source['id']==999994):
            continue
        server = 'http://www.atlashymenoptera.net/'
        # Récupérer le lien du fichier csv de la prochaine espèce
        dataset_url = server + source['link']
        #print("dataset_url for specie ", source['id']," : ",  dataset_url)
        # Charger le fichier csv de la prochaine espèce
        dataset = pd.read_csv(dataset_url, delimiter='\t', skiprows=2)
        dataset = dataset.drop(['Authors', 'Pub_year', 'Source'], axis=1)
        # Récupérer les données de ce CSV et les ajouter aux données de l'ensemble des espèces 
        cleaned_dataset += clean_dataset(dataset, source['id'])
        # Svg intérmédiaire
        #save_position_countries()
        print('------ finished', source['name'], 'id', source['id'], '---------')

    #displayContentByYear(cleaned_dataset)
    return pd.DataFrame(cleaned_dataset, columns=['Year', 'Latitude', 'Longitude', 'SpecieId', 'Country', 'Frequency'])

In [4]:

""" Vérifie si le format de l'année est correct ou non """
def invalide_year(data):    
    year=data[0]
    if(isinstance(year, float)):
        # Nombre déjà au foramt décimale (éviter un appel à strip() qui renvoie une exception)
        # Vérifier que la partie décimale est nulle
        yearInt = int(year)
        #print("invalide_year isflotat ", data, (yearInt  == year))
        #return (yearInt  == year) and yearInt > 0
    else :
        # Supprimer les espaces
        year2 = year.strip() 
        if year2 == '': return True
        if not str.isdigit(year2): return True
        yearInt = int(year2)
    # L'année doit être comprise entre 1800 et 2100
    if(yearInt>2100 or yearInt<1800): return True
    return False


def load_position_countries():
    mapPositionCountries = {}
    filename = 'pos_countries.csv'
    if(path.exists(filename)):
        with open(filename, newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                #rowData = row.split(",")
                #print(row, len(row))
                if(len(row)>=2 and (row[0]!="Latitude")):
                    pos = str(row[0]) + ',' + str(row[1])
                    if(row[2]!=''):
                        mapPositionCountries[pos] = (row[2]).strip()
            csvfile.close()            
            print("load_position_countries map size = ", len(mapPositionCountries))
    return mapPositionCountries
""" """

def save_position_countries():
    rows = []
    for position in mapPositionCountries:
        item = position.split(',')       
        country = mapPositionCountries[position]
        item.append(country)
        rows.append(item)
        #print("save_position_countries", position, country, item )
    print("save_position_countries map size = ", len(mapPositionCountries) )
    dataset = pd.DataFrame(rows, columns=['Latitude', 'Longitude', 'Country'])
    dataset.to_csv('pos_countries.csv', index=False)


def get_country(row):    
    #print("get_country begin", row, mapPositionCountries)
    pos = str(row[1]) + ',' + str(row[2])        
    if(not pos in mapPositionCountries):
        locations = geolocator.reverse(pos, language='en')
        if(locations!=None):
            try:
                country  = locations.raw['address']['country']
                print("get_country", pos, country)
                mapPositionCountries[pos] = country
                if(len(mapPositionCountries) % 100 == 0):
                    save_position_countries()
            except ValueError:                
                print("get_country error", ValueError)
    if(pos in mapPositionCountries):
        return mapPositionCountries[pos]
    print("### get_country  : country not found for position ", pos, " (maybe in the sea ...)")
    return ""


""" Récupération des données d'une espèce à partir du contenu du fichier csv """
def clean_dataset(dataset, specie_id):
    
    # Table des fréquences par année et position géographique et espèce
    frequencies_dict = {}

    for data in dataset.values:

        if invalide_year(data): continue
            
        # Clé d'une donnée élémentaire : année + position géogrpahique + identifiant de l'espèce
        key = '{} {} {} {}'.format(int(data[0]), data[1], data[2], specie_id)

        # Mise à jour de la fréquence associé à la clé
        if key in frequencies_dict.keys():
            # Clé déjà rencontrée dans le csv : on incrémente la fréquence associée
            frequencies_dict[key] += 1

        else: frequencies_dict[key] = 1

    frequencies = []

    for freq in frequencies_dict.keys():
        k = freq.split(' ')
        country = get_country(k)
        k.append(country)
        k.append(frequencies_dict[freq])
        frequencies.append(k)        
        #print(k)
    return frequencies
    # frequencies = pd.DataFrame(frequencies, columns=['Year', 'Latitude', 'Longitude', 'SpecieId', 'Frequency'])
    
    

In [5]:
mapPositionCountries = load_position_countries()


data_sources = get_all_data_sources()
dataset = fetch_all_dataset(data_sources)


dataset.to_csv('bombus_terrestris_freqs.csv.gz', index=False, compression='gzip')
with open('data_sources.json', 'w+') as output_file:
    json.dump(data_sources, output_file)
print('finished download')

load_position_countries map size =  16795
------ finished Bombus (Alpigenobombus) wurflenii id 1 ---------
------ finished Bombus (Alpinobombus) alpinus id 2 ---------
------ finished Bombus (Alpinobombus) balteatus id 3 ---------
------ finished Bombus (Alpinobombus) hyperboreus id 4 ---------
------ finished Bombus (Alpinobombus) polaris id 5 ---------
------ finished Bombus (Bombus) cryptarum id 6 ---------
### get_country  : country not found for position  49.90824,-4.81055  (maybe in the sea ...)
### get_country  : country not found for position  49.88551,-5.89353  (maybe in the sea ...)
### get_country  : country not found for position  50.09207,-4.53785  (maybe in the sea ...)
### get_country  : country not found for position  41.81879,50.15716  (maybe in the sea ...)
### get_country  : country not found for position  43.58747,35.84903  (maybe in the sea ...)
### get_country  : country not found for position  44.15164,37.374390000000005  (maybe in the sea ...)
### get_country  :

------ finished Bombus (Psithyrus) quadricolor id 38 ---------
------ finished Bombus (Psithyrus) rupestris id 39 ---------
### get_country  : country not found for position  56.535880000000006,12.03449  (maybe in the sea ...)
### get_country  : country not found for position  58.177519999999994,10.87037  (maybe in the sea ...)
------ finished Bombus (Psithyrus) sylvestris id 40 ---------
### get_country  : country not found for position  36.9471,52.46001  (maybe in the sea ...)
------ finished Bombus (Psithyrus) vestalis id 41 ---------
------ finished Bombus (Pyrobombus) brodmannicus id 42 ---------
------ finished Bombus (Pyrobombus) cingulatus id 43 ---------
------ finished Bombus (Pyrobombus) glacialis id 44 ---------
------ finished Bombus (Pyrobombus) haematurus id 45 ---------
### get_country  : country not found for position  57.46371,19.49969  (maybe in the sea ...)
------ finished Bombus (Pyrobombus) hypnorum id 46 ---------
### get_country  : country not found for position

### get_country  : country not found for position  53.862190000000005,-4.9768300000000005  (maybe in the sea ...)
### get_country  : country not found for position  50.28137,-2.5789299999999997  (maybe in the sea ...)
### get_country  : country not found for position  50.28137,-2.5789299999999997  (maybe in the sea ...)
### get_country  : country not found for position  52.07615,1.9786  (maybe in the sea ...)
### get_country  : country not found for position  56.56908000000001,-1.8608  (maybe in the sea ...)
### get_country  : country not found for position  53.862190000000005,-4.9768300000000005  (maybe in the sea ...)
### get_country  : country not found for position  53.8776,-3.4563300000000003  (maybe in the sea ...)
### get_country  : country not found for position  53.8776,-3.4563300000000003  (maybe in the sea ...)
### get_country  : country not found for position  53.50286,-4.96007  (maybe in the sea ...)
### get_country  : country not found for position  52.07615,1.9786  (mayb